if not yet available some libraries have to be installed :<br>
- gmsh (best installed globally through package management system)
- python3 -m pip install pygmsh --user
- python3 -m pip install vtk --user

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import vtk

import pygmsh
import numpy as np

In [24]:
def CircularMesh(R=1.0, ratio=1.0, lcar=0.1):
    """
    Create a circular mesh with radius R around the origin [0,0,0].
    The normal vector is (0,0,1) in z-direction, the disk lies in the x-y-plane.
    
    The density of the grid is increased by ratio in the x-direction.
    The average cell dimensions are given by lcar. The average cell dimension in
    x-direction, thus, is lcar/ratio.
    
    returns:
    - an array of point corrdinates [3 float]
    - a connection list of triangles [3 int references into the array of points]
    """
    geom = pygmsh.built_in.Geometry()
    # we create the initial geometry as a streched ellipse to create
    # different scaling lengths (cell sizes) along the different axes
    p1 = geom.add_point([ratio, 0.0, 0.0], lcar)
    p2 = geom.add_point([0.0, 1.0, 0.0], lcar)
    p3 = geom.add_point([-ratio, 0.0, 0.0], lcar)
    p4 = geom.add_point([0.0, -1.0, 0.0], lcar)
    pc = geom.add_point([0.0, 0.0, 0.0], lcar)
    pa = geom.add_point([1.0, 0.0, 0.0], lcar)
    # the patch is circumscribed with four elliptic arcs
    e1 = geom.add_ellipse_arc(start=p1, center=pc, point_on_major_axis=pa, end=p2)
    e2 = geom.add_ellipse_arc(start=p2, center=pc, point_on_major_axis=pa, end=p3)
    e3 = geom.add_ellipse_arc(start=p3, center=pc, point_on_major_axis=pa, end=p4)
    e4 = geom.add_ellipse_arc(start=p4, center=pc, point_on_major_axis=pa, end=p1)
    # these are combined into a line loop
    ll = geom.add_line_loop([e1,e2,e3,e4])
    geom.add_plane_surface(ll)
    # now we can create the mesh
    mesh = pygmsh.generate_mesh(geom, dim=2, verbose=False)
    # we reverse the streching by scaling the coordinates accordingly
    points = np.array([ p*R*[1.0/ratio,1,1] for p in mesh.points ])
    triangles = mesh.cells['triangle']
    return points, triangles

In [79]:
pts, tris = CircularMesh(R=2.0, ratio=1.0, lcar=0.1)
print("%d triangular segments" % len(tris))
points = vtk.vtkPoints()
for p in pts:
    points.InsertNextPoint(p)
cells = vtk.vtkCellArray()
centers = vtk.vtkPoints()
for t in tris:
    cells.InsertNextCell(3, t)
    c = (pts[t[0]] + pts[t[1]] + pts[t[2]]) / 3.0
    centers.InsertNextPoint(c)


858 triangular segments


In [81]:
colors = vtk.vtkNamedColors()

# create a visualization of the triangular mesh
meshData = vtk.vtkPolyData()
meshData.SetPoints(points)
meshData.SetPolys(cells)
meshMapper = vtk.vtkPolyDataMapper()
meshMapper.SetInputData(meshData)
meshActor = vtk.vtkActor()
meshActor.SetMapper(meshMapper)
meshActor.GetProperty().SetPointSize(5)
meshActor.GetProperty().SetColor(colors.GetColor3d("Red"))
meshActor.GetProperty().EdgeVisibilityOn()

# create a visualization of the mesh center points
verts = vtk.vtkCellArray()
centerData = vtk.vtkPolyData()
centerData.SetPoints(centers)
for i, t in enumerate(tris):
    verts.InsertNextCell(1, [i])
centerData.SetVerts(verts)
centerMapper = vtk.vtkPolyDataMapper()
centerMapper.SetInputData(centerData)
centerActor = vtk.vtkActor()
centerActor.SetMapper(centerMapper)
centerActor.GetProperty().SetPointSize(3)
centerActor.GetProperty().SetColor(colors.GetColor3d("Blue"))

# visualize the coordinate system
axesActor = vtk.vtkAxesActor()

# create a render window
renderer = vtk.vtkRenderer()
renderWindow = vtk.vtkRenderWindow()
renderWindow.SetSize(800,600)
renderWindow.AddRenderer(renderer)
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)
renderWindowInteractor.Initialize()
style = vtk.vtkInteractorStyleTrackballCamera()
renderWindowInteractor.SetInteractorStyle(style)

# add the actors to the scene
renderer.AddActor(meshActor)
renderer.AddActor(centerActor)
renderer.AddActor(axesActor)
renderer.SetBackground(colors.GetColor3d("SlateGray"))

# render and interact
renderWindow.Render()
renderWindowInteractor.Start()

# now the interaction is running until we close the window

# cleanup after closing the window
del renderWindow
del renderWindowInteractor